In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

import warnings
warnings.filterwarnings('ignore')

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization

Using TensorFlow backend.


In [2]:
num = re.compile(r'(\d+)')
def key_func(a):
    ids = num.split(a)
    ids[1::2] = map(int, ids[1::2])
    return ids

train = './nnfl-assignment-i/final_train/final_train'
test = './nnfl-assignment-i/final_test/final_test'
x_train=[]
for file in sorted(os.listdir(train),key=key_func):
    img='./nnfl-assignment-i/final_train/final_train/'+file
    image = cv2.imread(img)
    image = cv2.resize(image,(64,64))
    x_train.append(image)

x_train = np.asarray(x_train)

col=['File Name','Class']
data = pd.read_csv('nnfl-assignment-i/train_mappings_final.csv',names=col)
y_train = data.Class.tolist()
y_train = y_train[1:]
y_train=np.array(y_train[1:], dtype='int')
y_train = y_train.reshape((y_train.shape[0],1))

In [3]:
EPOCHS = 30
NUM_CLASSES = 2

In [4]:
x_train = x_train.astype('float32')
x_train /= 255

y_train = np_utils.to_categorical(y_train)

In [5]:
from keras.layers import LeakyReLU

In [6]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3)))
model.add(LeakyReLU(alpha=0.01))
model.add(Conv2D(32, (3, 3)))
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3)))
model.add(LeakyReLU(alpha=0.01))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
model.fit(x_train, y_train, validation_split=0.1, validation_data=None, shuffle=True, epochs=EPOCHS, batch_size=40)

In [23]:
model.save('model-2017A7PS0007G.h5')